## Overview
<a href="https://archive.ics.uci.edu/ml/datasets/online+retail">Online retail</a> is a transnational dataset which contains all the transactions occurring between 01/12/2010 and 09/12/2011 for a UK-based and registered non-store online retail. The company mainly sells unique all-occasion gifts. Many customers of the company are wholesalers.

## Source
UCI Machine Learning Repository: https://archive.ics.uci.edu/ml/datasets/online+retail

## Business Goal
To segment the Customers based on RFM so that the company can target its customers efficiently.

## Methodology

1. [Reading and Understanding the Data](#1) <br>
    a. Creating a Data Dictionary
2. [Data Cleaning](#2)
3. [Data Preparation](#3) <br>
    a. Scaling Variables
4. [Model Building](#4) <br>
    a. K-means Clustering <br>
    b. Finding the Optimal K
5. [Final Analysis](#5)

<a id="1"></a> <br>
### 1 : Reading and Understanding Data

In [ ]:
# import required libraries for dataframe and visualization

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt
import plotly as py 
import plotly.graph_objs as go

from plotly.offline import init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)

# import required libraries for clustering
import sklearn
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from scipy.cluster.hierarchy import linkage
from scipy.cluster.hierarchy import dendrogram
from scipy.cluster.hierarchy import cut_tree

In [30]:
# Reading the data on which analysis needs to be done
retail = pd.read_csv('dataset/OnlineRetail.csv', encoding='ISO-8859-1')

#### Data Dictionary 

First Header  | Definition    |  Description  | Data Type
------------- | ------------- | ------------- | -------------
InvoiceNo  | Invoice number | A 6-digit integral number uniquely assigned to each transaction. If this code starts with letter 'c', it indicates a cancellation. | Nominal
StockCode | Product (item) code | A 5-digit integral number uniquely assigned to each distinct product. | Nominal
Description | Product (item) name | Name of Product | Nominal
Quantity | Quantity | The quantities of each product (item) per transaction | Numeric
InvoiceDate | Invoice Date and time | The day and time when each transaction was generated. | Numeric
UnitPrice | Unit price | Product price per unit in sterling. | Numeric
CustomerID | Customer number | A 5-digit integral number uniquely assigned to each customer. | Nominal
Country | Country name | The name of the country where each customer resides. | Nominal

In [31]:
# shape of df


In [32]:
# df description


<a id="2"></a> <br>
### 2 : Data Cleaning

In [33]:
# Calculating the Missing Values % contribution in DF


In [34]:
# Droping rows having missing values


In [35]:
# Changing the datatype of Customer Id as per Business understanding


<a id="3"></a> <br>
### 3 : Data Preparation

#### Customers will be analyzed based on 3 factors:
- R (Recency): Number of days since last purchase
- F (Frequency): Number of tracsactions
- M (Monetary): Total amount of transactions (revenue contributed)

In [36]:
# New Attribute : Monetary


In [37]:
# New Attribute : Frequency


In [38]:
# Merging the two dfs


In [39]:
# New Attribute : Recency

# Convert to datetime to proper datatype


In [40]:
# Compute the maximum date to know the last transaction date



In [41]:
# Compute the difference between max date and transaction date


In [42]:
# Compute last transaction date to get the recency of customers


In [43]:
# Extract number of days only



In [44]:
# Merge tha dataframes to get the final RFM dataframe


Exploratory Data Analysis 

In [ ]:
# Histogram #1: Number of Transactions per Customer

transactions_per_customer = retail.groupby('CustomerID').size()

plt.figure(figsize=(10, 6))
plt.hist(transactions_per_customer, bins=50, edgecolor='black')

plt.title('Number of Transactions per Customer', fontsize=16)
plt.xlabel('Number of Transactions', fontsize=14)
plt.ylabel('Number of Customers', fontsize=14)

plt.show()

In [ ]:
# Histogram # 2
item_counts = retail['Description'].value_counts().sort_values(ascending=False).iloc[0:15]
plt.figure(figsize=(18,6))
sns.barplot(x=item_counts.index, y=item_counts.values, palette=sns.cubehelix_palette(15))
plt.ylabel("Counts")
plt.title("Which items were bought more often?")
plt.xticks(rotation=90)

In [ ]:
retail['TotalSales'] = retail['Quantity'] * retail['UnitPrice']

# Group by StockCode and calculate the total sales per product
sales_by_stockcode = retail.groupby('StockCode')['TotalSales'].sum()

sales_by_stockcode = sales_by_stockcode[sales_by_stockcode > 0]

plt.figure(figsize=(12, 6))

plt.subplot(1, 2, 1)  # 1 row, 2 columns, 1st subplot
plt.boxplot(sales_by_stockcode, vert=False, patch_artist=True, boxprops=dict(facecolor='skyblue', color='black'))
plt.title('Boxplot of Total Sales Value by StockCode', fontsize=16)
plt.xlabel('Total Sales Value', fontsize=14)
log_sales_by_stockcode = np.log(sales_by_stockcode)


plt.subplot(1, 2, 2)  
plt.boxplot(log_sales_by_stockcode, vert=False, patch_artist=True, boxprops=dict(facecolor='lightgreen', color='black'))
plt.title('Boxplot of Log-Transformed Total Sales Value by StockCode', fontsize=16)
plt.xlabel('Log(Total Sales Value)', fontsize=14)

plt.tight_layout()
plt.show()


In [ ]:
# Correlation heatmap

# Select relevant columns and drop any rows with NaN values
data = retail[['Quantity', 'UnitPrice']]

# Calculate the correlation matrix
correlation_matrix = data.corr()

# Set the size of the heatmap
plt.figure(figsize=(8, 6))

# Create the heatmap using Seaborn
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', center=0, square=True, linewidths=0.5)

# Add titles and labels
plt.title('Correlation Heatmap between Quantity and UnitPrice', fontsize=16)
plt.xticks(rotation=45, fontsize=12)
plt.yticks(rotation=45, fontsize=12)

# Show the heatmap
plt.tight_layout()
plt.show()


#### Rescaling the Attributes

It is extremely important to rescale the variables so that they have a comparable scale.<br>
There are two common ways of rescaling:

1. Min-Max scaling 
2. Standardization (mean-0, sigma-1) 

Here we execute Standard Scaling.

In [48]:
# Rescaling the attributes

# Instantiate

# fit_transform


## <span style="color: red;">Execute MinMax Scaling in the next box</span> 

<a id="4"></a> <br>
### 4 : Building the Model

### K-Means Clustering

K-means clustering is one of the simplest and popular unsupervised machine learning algorithms.<br>

The algorithm works as follows:

- First we initialize k points, called means, randomly.
- We categorize each item to its closest mean and we update the mean’s coordinates, which are the averages of the items categorized in that mean so far.
- We repeat the process for a given number of iterations and at the end, we have our clusters.

In [49]:
# k-means with some arbitrary k


In [50]:
#create a K_means function here


In [51]:
#plot your clusters


## <span style="color: red;">Finding the Optimal Number of Clusters</span> 

#### Elbow Curve to get the right number of Clusters
A fundamental step for any unsupervised algorithm is to determine the optimal number of clusters into which the data may be clustered. The Elbow Method is one of the most popular methods to determine this optimal value of k.

## <span style="color: red;">Box Plots of Clusters created</span> 

<a id="5"></a> <br>
## Step 5 : Final Analysis

## <span style="color: red;">Findings</span> 

#### Student Name: